In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time

BASE_URL = "https://publicdomainmovie.net"

def get_movie_links():
    movie_links = []
    base_url = "https://publicdomainmovie.net"
    total_pages = 80

    for page in range(total_pages):
        url = f"{base_url}/home?page={page}"
        
        response = requests.get(url)
        if response.status_code != 200:
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        movie_rows = soup.select('div.views-row')

        for row in movie_rows:
            cover_div = row.select_one('div.cover')
            if cover_div:
                link_tag = cover_div.find('a', href=True)
                if link_tag:
                    link = link_tag['href']
                    if not link.startswith('http'):
                        link = base_url + link
                    movie_links.append(link)
        
        time.sleep(1)

    return movie_links

def download_movie_from_page(movie_page_url, download_folder="downloads"):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    response = requests.get(movie_page_url)
    if response.status_code != 200:
        print(f"Failed to retrieve: {movie_page_url}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    download_div = soup.find('div', class_='field-name-download')
    if not download_div:
        print(f"No direct download link found on page: {movie_page_url}")
        return

    download_link = download_div.find('a', href=True)
    if not download_link:
        print(f"No download link found in field-name-download on page: {movie_page_url}")
        return

    movie_download_url = download_link['href']
    if not movie_download_url.startswith('http'):
        movie_download_url = BASE_URL + movie_download_url

    filename = movie_page_url.rstrip('/').split('/')[-1] + ".mp4"
    filepath = os.path.join(download_folder, filename)

    print(f"Downloading {movie_download_url} to {filepath}")
    with requests.get(movie_download_url, stream=True) as r:
        if r.status_code == 200:
            with open(filepath, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print(f"Downloaded: {filepath}")
        else:
            print(f"Failed to download MP4 from: {movie_download_url}")

# Main
movie_urls = get_movie_links()
print(len(movie_urls))

for url in movie_urls:
    download_movie_from_page(url)
    time.sleep(1)


718


KeyboardInterrupt: 